In [1]:
from ape import Contract, chain, networks,accounts,project,convert,api
from web3 import Web3, AsyncWeb3
import json


w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.is_connected()

w3.eth.default_account = w3.eth.accounts[0]
print("default account balance:",w3.eth.default_account, w3.eth.get_balance(w3.eth.default_account))

default account balance: 0x93dfBbF293297F30E68FD287b3fB3E21a2450887 999998343143903076444


In [2]:
with networks.parse_network_choice("ethereum:local:ganache") as provider:
    js = project.CF.contract_type.dict()
    # # ["abi"]
    # print(js)
    # print(list(js.keys()))


ExampleContract = w3.eth.contract(abi=js["abi"], bytecode=js["deploymentBytecode"]["bytecode"])
tx_hash = ExampleContract.constructor(w3.to_wei(3,"ether")).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)


contract = w3.eth.contract(address=tx_receipt.contractAddress, abi=js["abi"])

contract.functions.isOwner().call()

INFO: Connecting to existing 'Ganache' process.


'0x93dfBbF293297F30E68FD287b3fB3E21a2450887'

In [3]:
# Get deployment bytecode
# deploy_bytecode = w3.eth.contract(abi=[]).constructor().buildTransaction({'data': contract_bytecode})['data']

# Estimate gas 
# gas_estimate = w3.eth.estimate_gas({
#   'from': w3.eth.accounts[0], 
#   'data': js["deploymentBytecode"]["bytecode"]
# })
# w3.eth.estimate_gas(tx_receipt)
ExampleContract.constructor(1).estimate_gas()

775737

In [4]:
import pprint

amount = 1
# account = "0xDc541B8b0e326bdf734744f497713A7cb2255b0F"
# private_key = "0x0aa622a36df1c7db6007e8d03f7b42fe1d7224cfc1cd1fc6cf7351003c70a668"
# w3.eth.default_account = account

# nonce = w3.eth.get_transaction_count(w3.eth.accounts[1])

contributors = [w3.eth.accounts[1],w3.eth.accounts[2]]

for contributor in contributors:

    tx = {
            "chainId": w3.eth.chain_id,  # Replace with the chain ID of your network
            "from": contributor,
            # "to": tx_receipt.contractAddress,
            "value": w3.to_wei(amount, "ether"),
            # "gas": gas,
            # "gasPrice": gasPrice,
            "gas": 200000,  # Specify the gas limit
            "gasPrice": w3.to_wei("20", "gwei"),  # Specify the gas price
            # "nonce": nonce,
        }

    tx_hash = contract.functions.contribute().transact(tx)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    pprint.pprint(dict(tx_receipt))

# transaction = contract.functions.contribute().build_transaction(tx)

# signed_transaction = w3.eth.account.sign_transaction(transaction, private_key)
# transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# # Render the result template with the transaction hash
# transaction_hash=transaction_hash.hex()
# transaction_hash

{'blockHash': HexBytes('0x4375f993929ce0077c5ef5d899bf9db90435bd1e7ef80aa63c38bf7ebb41c38c'),
 'blockNumber': 7,
 'contractAddress': None,
 'cumulativeGasUsed': 137790,
 'effectiveGasPrice': 20000000000,
 'from': '0xC62460B42C3502bd7CBd98644ABCcB4e0F968E23',
 'gasUsed': 137790,
 'logs': [AttributeDict({'address': '0xED2e531EC706b0E914daDB7CC99a5F5449493c00', 'blockHash': HexBytes('0x4375f993929ce0077c5ef5d899bf9db90435bd1e7ef80aa63c38bf7ebb41c38c'), 'blockNumber': 7, 'data': HexBytes('0x000000000000000000000000c62460b42c3502bd7cbd98644abccb4e0f968e230000000000000000000000000000000000000000000000000de0b6b3a7640000'), 'logIndex': 0, 'removed': False, 'topics': [HexBytes('0x23bac35767dec08bf219a302474bbc6687dfaaf3776a62d45bb7811de040e26d')], 'transactionHash': HexBytes('0x393d98a2858014b6e8ac558fccc855b993f3e5be861434101ecc388933bc0abb'), 'transactionIndex': 0})],
 'logsBloom': HexBytes('0x0000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000000

In [5]:
try:
    tx_hash=contract.functions.completeProject().transact()
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(tx_receipt)
except Exception as e:
    print(e)

execution reverted: VM Exception while processing transaction: revert Funding goal has not been reached.


In [6]:
# Example: Check crowdfunding status
def check_crowdfunding_status():
    project_owner = contract.functions.projectOwner().call()
    goal_amount = contract.functions.goalAmount().call()
    total_funding = contract.functions.totalFunding().call()
    funding_goal_reached = contract.functions.fundingGoalReached().call()
    project_completed = contract.functions.projectCompleted().call()

    print("contractAddress:", contract.address)
    print(f"Project Owner: {project_owner}")
    print(f"Default Account: {w3.eth.default_account}")
    print(f"Goal Amount: {goal_amount} Wei")
    print(f"Goal Amount: {w3.from_wei(goal_amount,'ether')} Ether")
    print(f"Total Funding: {total_funding} Wei")
    print(f"Total Funding: {w3.from_wei(total_funding,'ether')} Ether")

    print(f"Funding Goal Reached: {funding_goal_reached}")
    print(f"Project Completed: {project_completed}")
    print("getContributorCount: ", contract.functions.getContributorCount().call())
    # print("getContributorAtIndex:",contract.functions.getContributors().call())
    (
        contributor_addresses,
        contribution_amounts,
        contribution_datetime,
    ) = contract.functions.getContributors().call()
    print("contributors:::::")
    for i in range(len(contributor_addresses)):
        # contributors_dict[contributor_addresses[i].hex()] = contribution_amounts[i]
        print(
            contributor_addresses[i], contribution_amounts[i], contribution_datetime[i]
        )


check_crowdfunding_status()

contractAddress: 0xED2e531EC706b0E914daDB7CC99a5F5449493c00
Project Owner: 0x93dfBbF293297F30E68FD287b3fB3E21a2450887
Default Account: 0x93dfBbF293297F30E68FD287b3fB3E21a2450887
Goal Amount: 3000000000000000000 Wei
Goal Amount: 3 Ether
Total Funding: 2000000000000000000 Wei
Total Funding: 2 Ether
Funding Goal Reached: False
Project Completed: False
getContributorCount:  2
contributors:::::
0xC62460B42C3502bd7CBd98644ABCcB4e0F968E23 1000000000000000000 1699894585
0xdc33a23979C235Fbb028006ED11659fC98444262 1000000000000000000 1699894585


In [7]:
try:
    tx_hash=contract.functions.refundContributors().transact()
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    pprint.pprint(dict(tx_receipt))
except Exception as e:
    print(e)

{'blockHash': HexBytes('0x1ebdbf67064e341d8187754f64f90a7b3693d8279aa0f94a490ba9c278cb4d5b'),
 'blockNumber': 9,
 'contractAddress': None,
 'cumulativeGasUsed': 61222,
 'effectiveGasPrice': 391036493,
 'from': '0x93dfBbF293297F30E68FD287b3fB3E21a2450887',
 'gasUsed': 61222,
 'logs': [AttributeDict({'address': '0xED2e531EC706b0E914daDB7CC99a5F5449493c00', 'blockHash': HexBytes('0x1ebdbf67064e341d8187754f64f90a7b3693d8279aa0f94a490ba9c278cb4d5b'), 'blockNumber': 9, 'data': HexBytes('0x000000000000000000000000c62460b42c3502bd7cbd98644abccb4e0f968e230000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000000000065525539'), 'logIndex': 0, 'removed': False, 'topics': [HexBytes('0x0754663d79ddf47e96611d36d595a38ce0918960aa5965913d75c482d0477ed6')], 'transactionHash': HexBytes('0xb2a16c7856c0d2aad098019eb306bdfe4765728e21a9ce6062749ef1c06c3ccb'), 'transactionIndex': 0}),
          AttributeDict({'address': '0xED2e531EC706b0E914daDB7CC99a

In [8]:
# tx_hash=contract.functions.withdrawFunds().transact()
# tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
# tx_receipt

In [9]:
# tx_hash=contract.functions.contributions().call()
# tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
# tx_receipt

In [10]:
tx_receipt["logs"][0]

AttributeDict({'address': '0xED2e531EC706b0E914daDB7CC99a5F5449493c00',
 'blockHash': HexBytes('0x1ebdbf67064e341d8187754f64f90a7b3693d8279aa0f94a490ba9c278cb4d5b'),
 'blockNumber': 9,
 'data': HexBytes('0x000000000000000000000000c62460b42c3502bd7cbd98644abccb4e0f968e230000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000000000065525539'),
 'logIndex': 0,
 'removed': False,
 'topics': [HexBytes('0x0754663d79ddf47e96611d36d595a38ce0918960aa5965913d75c482d0477ed6')],
 'transactionHash': HexBytes('0xb2a16c7856c0d2aad098019eb306bdfe4765728e21a9ce6062749ef1c06c3ccb'),
 'transactionIndex': 0})

In [11]:
tx_receipt["logs"][0]["transactionHash"].hex()

'0xb2a16c7856c0d2aad098019eb306bdfe4765728e21a9ce6062749ef1c06c3ccb'

In [12]:
contract.events.RefundedContributor().get_logs()

(AttributeDict({'args': AttributeDict({'contributor': '0xC62460B42C3502bd7CBd98644ABCcB4e0F968E23',
   'amount': 1000000000000000000,
   'timestamp': 1699894585}),
  'event': 'RefundedContributor',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xb2a16c7856c0d2aad098019eb306bdfe4765728e21a9ce6062749ef1c06c3ccb'),
  'address': '0xED2e531EC706b0E914daDB7CC99a5F5449493c00',
  'blockHash': HexBytes('0x1ebdbf67064e341d8187754f64f90a7b3693d8279aa0f94a490ba9c278cb4d5b'),
  'blockNumber': 9}),
 AttributeDict({'args': AttributeDict({'contributor': '0xdc33a23979C235Fbb028006ED11659fC98444262',
   'amount': 1000000000000000000,
   'timestamp': 1699894585}),
  'event': 'RefundedContributor',
  'logIndex': 1,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xb2a16c7856c0d2aad098019eb306bdfe4765728e21a9ce6062749ef1c06c3ccb'),
  'address': '0xED2e531EC706b0E914daDB7CC99a5F5449493c00',
  'blockHash': HexBytes('0x1ebdbf67064e341d8187754f64f90a7b3693d8279aa0f94a490ba9c2